In [1]:
from bs4 import BeautifulSoup
import requests
import json
from tqdm import tqdm
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
base = "https://gg.deals"
addr1 = "/games/?sort=metascore&type=1&page="

In [3]:
lst_url = []

In [4]:
for i in range(7):
    link = base + addr1 + '{}'.format(i+1)
    req = requests.get(link)
    soup = BeautifulSoup(req.text, 'html.parser')
    urls = soup.find_all('a', class_='full-link')
    if i < 6:
        for u in urls:
            lst_url.append(u['href'])
    else:
        for j in range(12):
            lst_url.append(urls[j]['href'])


In [5]:
len(lst_url)

300

In [6]:
lst_url[0]

'/game/disco-elysium/'

In [7]:
base + lst_url[0]

'https://gg.deals/game/disco-elysium/'

In [8]:
def parse(url):
    #driver = webdriver.Chrome(ChromeDriverManager().install())
    driver = webdriver.Chrome('C:\\Users\\Иван\\.wdm\\drivers\\chromedriver\\win32\\101.0.4951.41\\chromedriver.exe')
    driver.get(base + url)
    driver.maximize_window()
    driver.find_element_by_xpath('/html/body/div[1]/div/a').click()
    try:
        driver.find_element_by_xpath('//*[@id="game-dlcs"]/div[2]/div[2]/a').click()
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="game-packs"]/div[2]/div[2]/a').click()
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="game-info-tags"]/div/button').click()
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="game-info-features"]/div/button').click()
    except:
        pass
    
    ps = driver.page_source
    soup = BeautifulSoup(ps, 'html.parser')
    driver.quit()
    
    card = {}
    card['url'] = base + url
    id_ = soup.find('div', class_='game-card').find('div', class_='game-header game-header-container container').find('div', class_='game-info-actions game-single-actions').div['data-game-id']
    name = soup.find('div', class_='breadcrumbs-widget pjax-inner-replace').find('a', class_='active')
    card['name'] = name.text.split(sep='\n')[1]
    image = soup.find('div', class_='game-card').find('div', class_='game-header game-header-container container').find('div', class_='game-info-image').find('img', class_='image-blur')
    if (image != None):
        card['image'] = image['src']
    market_url = soup.find('div', class_='game-card').find('div', class_='game-header game-header-container container').find('div', class_='game-info-image').find('a', class_='game-link-widget')
    if (market_url != None):
        card['market_url'] = market_url['href']
    counts = soup.find('div', class_='game-card').find('div', class_='game-header game-header-container container').find_all('span', class_='count')
    if (counts != None):
        card['wishlist_count'] = counts[0].text
        if (len(counts) > 2):
            card['alert_count'] = counts[2].text
        if (len(counts) > 4):
            card['owners_count'] = counts[4].text
    col_right = soup.find('div', class_='game-card').find('div', class_='game-section game-offers').find('div', class_='col-right')
    release_date = col_right.find('div', class_='game-info-details').find_all('p')[0].text
    developer = col_right.find('div', class_='game-info-details').find_all('p')[1].text
    if (release_date != None):
        card['release_date'] = release_date
    if (developer != None):
        card['developer'] = developer
    scores = col_right.find('div', class_='game-info-details-content').find_all('div', class_='score')
    metacritic_score = scores[0].find_all('span', class_='overlay')[0].text
    user_score = scores[0].find_all('span', class_='overlay')[1].text
    if (metacritic_score != None):
        card['metacritic_score'] = metacritic_score
    if (user_score != None):
        card['user_score'] = user_score
    if (len(scores) > 1):
        review_label_and_count = scores[1].find('span', class_='reviews-label').text
        if (review_label_and_count != None):
            rew = review_label_and_count
            rew = rew.replace(')', ' (')
            rew = rew.split(sep=' (')
            card['review_label'] = rew[0]
            card['review_count'] = rew[1]
        review_positive_pctg = scores[1].find('span', class_='reviews-label')['data-original-title']
        if (review_positive_pctg != None):
            card['review_positive_pctg'] = review_positive_pctg.split(sep='%')[0]
    genres_lst = col_right.find('div', id='game-info-genres').find_all('a', class_='badge-wrapper tag badge')
    if (genres_lst != None):
        genres = [item.text for item in genres_lst]
        card['genres'] = genres
    tags_lst = col_right.find('div', id='game-info-tags').find_all('a', class_='badge-wrapper tag badge')
    if (tags_lst != None):
        tags = [item.text for item in tags_lst]
        card['tags'] = tags
    features_lst = col_right.find('div', id='game-info-features')
    if (features_lst != None):
        features_lst = features_lst.find_all('a', class_='badge-wrapper tag badge')
        if (features_lst != None):
            features = [item.text for item in features_lst]
            card['features'] = features
    platforms_lst = col_right.find('div', class_='game-info-details').find_all('a', class_='platform-link-icon platform-icon')
    if (platforms_lst != None):
        platforms = [p.svg['data-original-title'] for p in platforms_lst]
        card['platforms'] = platforms
    col_left = soup.find('div', class_='game-card').find('div', class_='game-section game-offers').find('div', class_='col-left')
    dlcs_ = col_left.find('section', id='game-dlcs')
    packs_ = col_left.find('section', id='game-packs')
    if (dlcs_ != None):
        dlcs_lst = dlcs_.find_all('a', class_='full-link')
        dlcs = [base + i['href'] for i in dlcs_lst]
        card['dlcs'] = dlcs
    else:
        card['dlcs'] = []
    if (packs_ != None):
        packs_lst = packs_.find_all('a', class_='full-link')
        packs = [base + i['href'] for i in packs_lst]
        card['packs'] = packs
    else:
        card['packs'] = []
    
    req = requests.get("https://gg.deals/us/games/chartHistoricalData/" + str(id_) + "/?showKeyshops=1", headers={'x-requested-with': 'XMLHttpRequest'})
    if (req != None):
        dct = req.json()
        if ('chartData' in dct.keys()):
            lst = []
            for i in range(len(dct['chartData']['deals'])):
                ts = dct['chartData']['deals'][i]['x']
                price = dct['chartData']['deals'][i]['y']
                shop = dct['chartData']['deals'][i]['shop']
                lst.append({'ts': ts, 'price': price, 'shop': shop})
            card['price_history'] = lst
    
    return card

In [11]:
parse(lst_url[1])

C:\Users\3C8A~1\AppData\Local\Temp/ipykernel_21180/164344019.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\\Users\\Иван\\.wdm\\drivers\\chromedriver\\win32\\101.0.4951.41\\chromedriver.exe')
C:\Users\3C8A~1\AppData\Local\Temp/ipykernel_21180/164344019.py:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[1]/div/a').click()
C:\Users\3C8A~1\AppData\Local\Temp/ipykernel_21180/164344019.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="game-dlcs"]/div[2]/div[2]/a').click()
C:\Users\3C8A~1\AppData\Local\Temp/ipykernel_21180/164344019.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[

{'url': 'https://gg.deals/game/grand-theft-auto-v/',
 'name': 'Grand Theft Auto V',
 'image': 'https://img.gg.deals/d6/17/cd1d2498782e3a4eff30b5e11581780d8a76_307xt176.jpg',
 'market_url': 'https://gg.deals/redirect/d5e1410d9f8751add1432d420f68f154bc516535/?utm_source=games%2Fsingle',
 'wishlist_count': '4381',
 'alert_count': '723',
 'owners_count': '16633',
 'release_date': '14 Apr 2015',
 'developer': 'Rockstar North / Rockstar Games',
 'metacritic_score': '96',
 'user_score': '7.8',
 'review_label': 'Very Positive',
 'review_count': '1,215,694',
 'review_positive_pctg': '84',
 'genres': ['Action', 'Adventure'],
 'tags': ['Action',
  'Adventure',
  'Atmospheric',
  'Automobile Sim',
  'Co-op',
  'Comedy',
  'Crime',
  'First-Person',
  'Funny',
  'Great Soundtrack',
  'Mature',
  'Moddable',
  'Multiplayer',
  'Open World',
  'Racing',
  'Sandbox',
  'Shooter',
  'Singleplayer',
  'Third Person',
  'Third-Person Shooter',
  'GTA V'],
 'features': ['Full controller support', 'Online 

In [287]:
results = []

In [288]:
import gzip
import json
import codecs

from multiprocessing.dummy import Pool, Queue

queue = Queue()   # очередь ссылок на книги
for url in lst_url:
    queue.put(url)


def process_page_wrapper(i):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(i), mode='wb+') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            try:
                record = parse(queue.get())
                results.append(record)
                record_str = json.dumps(record, ensure_ascii=False)
                print(record_str, file=f_json)
            except Exception as exc:
                print(exc)
            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)


with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))
    time.sleep(3)
    pool.join()
    pool.close()

  0%|                                                                                          | 0/300 [00:00<?, ?it/s]C:\Users\3C8A~1\AppData\Local\Temp/ipykernel_20864/164344019.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\\Users\\Иван\\.wdm\\drivers\\chromedriver\\win32\\101.0.4951.41\\chromedriver.exe')
C:\Users\3C8A~1\AppData\Local\Temp/ipykernel_20864/164344019.py:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[1]/div/a').click()
C:\Users\3C8A~1\AppData\Local\Temp/ipykernel_20864/164344019.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="game-dlcs"]/div[2]/div[2]/a').click()
C:\Users\3C8A~1\AppData\Local\Temp/ipykernel_20864/164344019.py:12: DeprecationWarning: find_element

list index out of range


 68%|██████████████████████████████████████████████████████▍                         | 204/300 [08:19<03:00,  1.88s/it]

list index out of range


 89%|██████████████████████████████████████████████████████████████████████▉         | 266/300 [10:57<01:13,  2.16s/it]

list index out of range


100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [12:08<00:00,  2.43s/it]


ValueError: Pool is still running

In [289]:
len(results)

297

In [290]:
# 3 не спарсились, не успел найти, где ошибка

In [301]:
results[7]

{'url': 'https://gg.deals/game/grand-theft-auto-v/',
 'name': 'Grand Theft Auto V',
 'image': 'https://img.gg.deals/d6/17/cd1d2498782e3a4eff30b5e11581780d8a76_307xt176.jpg',
 'market_url': 'https://gg.deals/redirect/d5e1410d9f8751add1432d420f68f154bc516535/?utm_source=games%2Fsingle',
 'wishlist_count': '4358',
 'alert_count': '723',
 'owners_count': '16504',
 'release_date': '14 Apr 2015',
 'developer': 'Rockstar North',
 'metacritic_score': '96',
 'user_score': '7.8',
 'review_label': 'Very Positive',
 'review_count': '1,213,208',
 'review_positive_pctg': '84',
 'genres': ['Action', 'Adventure'],
 'tags': ['Open World',
  'Action',
  'Multiplayer',
  'Automobile Sim',
  'Crime',
  'Third Person',
  'First-Person',
  'Shooter',
  'Adventure',
  'Mature',
  'Singleplayer',
  'Third-Person Shooter',
  'Racing',
  'Co-op',
  'Atmospheric',
  'Sandbox',
  'Funny',
  'Great Soundtrack',
  'Comedy',
  'Moddable'],
 'features': ['Single-player',
  'Online PvP',
  'Online Co-op',
  'Steam Ach